# Load data in

In [ ]:
import IPython
import matplotlib.pyplot as plt
import numpy as np
import soundfile as sf
from tqdm import tqdm

In [ ]:
import os
from pathlib import Path

current_dir = Path('./')
data_dir = current_dir / 'speech_commands_v0.01'
train_file_path = data_dir / "train_digit_list.txt"
test_file_path = data_dir / "testing_digit_list.txt"

train_file = open(train_file_path, "r")
training_list = [data_dir / x for x in train_file.read().splitlines()]

test_file = open(test_file_path, "r")
testing_list = [data_dir / x for x in test_file.read().splitlines()]

In [ ]:
import os
from pathlib import Path
import numpy as np

with open('X_train_original.npy', 'rb') as f:
    X_train_org = np.load(f)

with open('X_train_reverb_random.npy', 'rb') as f:
    X_train_reverb = np.load(f)
    
with open('X_test_reverb_random.npy', 'rb') as f:
    X_test_reverb = np.load(f)

In [ ]:
import librosa
from nara_wpe.wpe import wpe_v7
from nara_wpe.wpe import get_power
from nara_wpe.utils import stft, istft, get_stft_center_frequencies
from nara_wpe import project_root
import scipy.io.wavfile as wav

sample_num = 25
wav.write('./audio/wpe_original.wav',  16000, X_train_org[sample_num].astype(np.int16))
wav.write('./audio/wpe_reverb.wav', 16000, X_train_reverb[sample_num].astype(np.int16))

# Do filter for one signal

In [ ]:
delay = 3
iterations = 5
taps = 10

o, sr = librosa.load('./audio/wpe_original.wav', sr=16000)
y, sample_rate = librosa.load('./audio/wpe_reverb.wav', sr=16000)
Y = librosa.stft(y, n_fft=512, hop_length=128)
Z = wpe_v7(Y, taps=taps, delay=delay, iterations=iterations, statistics_mode='full')
z = librosa.istft(Z, hop_length=128)
sf.write('./audio/wpe_fix.wav', z, 16000)

In [ ]:
import librosa

def plot_stft(prefix):
    original_file = './audio/{}_original.wav'.format(prefix)
    reverb_file = './audio/{}_reverb.wav'.format(prefix)
    denoise_file = './audio/{}_fix.wav'.format(prefix)
    
    o = librosa.load(original_file, 16000)[0]
    r = librosa.load(reverb_file, 16000)[0]
    d = librosa.load(denoise_file, 16000)[0]
        
    O = librosa.amplitude_to_db(librosa.stft(o), ref=np.max)
    R = librosa.amplitude_to_db(librosa.stft(r), ref=np.max)
    D = librosa.amplitude_to_db(librosa.stft(d), ref=np.max)
    
    return O, R, D

In [ ]:
o, r, d = plot_stft('wpe')

In [ ]:
import librosa.display

plt.figure(figsize=(6, 4))
librosa.display.specshow(d, y_axis='log')
plt.colorbar(format='%+2.0f dB')
plt.title('Weighted Prediction Error')
plt.savefig('img/wpe_fix.png')
plt.show()

# Do filter for train signal

In [ ]:
train_data = []
for i in tqdm(range(len(X_train_reverb))):
    signal = librosa.stft(X_train_reverb[i].flatten(), n_fft=512, hop_length=128)
    Z = wpe_v7(signal, taps=taps, delay=delay, iterations=iterations, statistics_mode='full')
    z = librosa.istft(Z, hop_length=128)
    train_data.append(z)
    
train_data = np.asarray(train_data)

In [ ]:
save_dir = Path('./E2E_data')
np.save(save_dir / "WPE_train_reverb.npy", train_data)

# Do filter for test signal

In [ ]:
test_data = []
for i in tqdm(range(len(X_test_reverb))):
    signal = librosa.stft(X_test_reverb[i].flatten(), n_fft=512, hop_length=128)
    Z = wpe_v7(signal, taps=taps, delay=delay, iterations=iterations, statistics_mode='full')
    z = librosa.istft(Z, hop_length=128)
    test_data.append(z)
    
test_data = np.asarray(test_data)

In [ ]:
save_dir = Path('./E2E_data')
np.save(save_dir / "WPE_test_reverb.npy", test_data)